In [14]:
import requests
import pandas
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.supabase import SupabaseVectorStore
from supabase.client import Client, create_client

import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
papersDF = pandas.read_csv('csv-microRNAAN-set (1).csv')

In [3]:
for index, row in papersDF.iterrows():
    doi = row['DOI']
    pass

In [4]:
test_doi = "10.1038/s41584-020-0426-0"

In [9]:
req = requests.get(f"https://api.unpaywall.org/v2/{test_doi}?email=ryandu9221@gmail.com")
url = req.json().get('best_oa_location').get('url')
fileloader = OnlinePDFLoader(url)
loader = UnstructuredPDFLoader(fileloader.file_path)
splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 500,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = False,
)
chunks = loader.load_and_split(text_splitter=splitter)

In [10]:
chunks[0].metadata

{'source': '/var/folders/8d/pdk79qrd6db6nb4r13lqsqqc0000gn/T/tmpuqfme3sl/tmp.pdf'}

In [18]:
embedding = OpenAIEmbeddings()
supabase: Client = create_client(supabase_url=os.environ.get("SUPABASE_URL"), supabase_key=os.environ.get("SUPABASE_SERVICE_KEY"))

In [7]:
embedded = SupabaseVectorStore.from_documents(documents=chunks, embedding=embedding)

ProgrammingError: (psycopg2.errors.UndefinedObject) type "vector" does not exist
LINE 4:  embedding VECTOR, 
                   ^

[SQL: 
CREATE TABLE langchain_pg_embedding (
	collection_id UUID, 
	embedding VECTOR, 
	document VARCHAR, 
	cmetadata JSON, 
	custom_id VARCHAR, 
	uuid UUID NOT NULL, 
	PRIMARY KEY (uuid), 
	FOREIGN KEY(collection_id) REFERENCES langchain_pg_collection (uuid) ON DELETE CASCADE
)

]
(Background on this error at: https://sqlalche.me/e/20/f405)